Nominatim Geocoding

In [2]:
%pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [geopy]
Note: you may need to restart the kernel to use updated packages.


In [10]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Initialize geocoder with custom User-Agent (required)
geolocator = Nominatim(user_agent="geoencoding_app")

# Add a rate limiter to avoid being blocked
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1.1)

# Example coordinates
latitude = 19.0488479705527
longitude = 72.9065413679928

# Get location
location = reverse((latitude, longitude), exactly_one=True)

# Print full address
print(location.address)


Msrtc Bus Stop Chembur maitri park, Road Number 5, Maitri Park, M/W Ward, Zone 5, Mumbai Suburban, Maharashtra, 400089, India


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

df = pd.read_csv("sample1.csv")

geolocator = Nominatim(user_agent="reverse-geocoder-script")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)


def get_address(row):
    try:
        location = reverse((row['PrvLatitude'], row['PrvLongitude']))
        return location.address if location else "Not found"
    except:
        return "Error"


df["ReverseGeocodedAddress"] = df.apply(get_address, axis=1)
df.to_csv("sample1_with_addresses.csv", index=False)

print("Reverse geocoding complete. File saved as 'sample1_with_addresses.csv'")


✅ Reverse geocoding complete. File saved as 'sample1_with_addresses.csv'


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic


df = pd.read_csv("sample1.csv")

geolocator = Nominatim(user_agent="reverse-geocoder-script")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)
forward = RateLimiter(geolocator.geocode, min_delay_seconds=1)


def get_address_and_roundtrip(row):
    try:
        
        location = reverse((row['PrvLatitude'], row['PrvLongitude']))
        if location is None:
            return pd.Series(["Not found", None, None, None])
        
        address = location.address

        
        geo_location = forward(address)
        if geo_location is None:
            return pd.Series([address, None, None, None])

        roundtrip_lat = geo_location.latitude
        roundtrip_lon = geo_location.longitude

        
        original_coords = (row['PrvLatitude'], row['PrvLongitude'])
        new_coords = (roundtrip_lat, roundtrip_lon)
        distance_meters = geodesic(original_coords, new_coords).meters

        return pd.Series([address, roundtrip_lat, roundtrip_lon, distance_meters])

    except Exception as e:
        return pd.Series(["Error", None, None, None])


df[['ReverseGeocodedAddress', 'RoundTripLatitude', 'RoundTripLongitude', 'DistanceFromOriginalMeters']] = df.apply(get_address_and_roundtrip, axis=1)
df.to_csv("sample1_with_roundtrip_check.csv", index=False)

print("Round-trip geocoding complete. File saved as 'sample1_with_roundtrip_check.csv'")


✅ Round-trip geocoding complete. File saved as 'sample1_with_roundtrip_check.csv'
